# Algorytmy macierzowe - zadanie nr 2 - Eliminacja Gaussa i LU Faktoryzacja dla macierzy gęstych

"Proszę wybrać język programowania wedle uznania
Proszę napisać procedurę [S]=Schur_Complement(A,n,m)
gdzie A to macierz wejściowa, n to rozmiar tej macierzy A,
m to rozmiar podmacierzy (tzw. dopełnienia Schura),
powstałej poprzez wyeliminowanie n-m wierszy i kolumn
z macierzy A, wykorzystując [zatrzymując po n-m krokach]:
 
6. Faktoryzacja Cholesky’ego “wektorową” (slajd 28) 
"

Marcin Hawryluk, Norbert Wolniak <br>grupa: piątek 12:50B <hr>

In [156]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import pandas as pd

Do implementacji wybraliśmy język Python 3 wraz z biblioteką do obliczeń numerycznych numpy, która pozwala operować na macierzach zaimplementowanych bezpośrednio w języku C.

## Generowanie macierzy

Macierze, których będziemy używać, wygenerowaliśmy za pomocą dostarczonej procedury massmatrix, napisanej w środowisku Octave. Macierze zapisaliśmy w postaci pliku tekstowego, a następnie odczytaliśmy w Pythonie za pomocą poniższej funkcji.

In [157]:
def read_matrix(file_name):
    with open(file_name, 'r') as file:
        for line in file:
            if line.strip() == '':
                continue
            if line[0] == '#':
                if line[2:6] == "rows":
                    _, _, size = line.split()
                    size = int(size)
                    matrix = np.zeros((size, size))
            else:
                row, col, val = line.split(' ')
                matrix[int(row)-1, int(col)-1] = val
            
    return matrix

In [158]:
matrix_small = read_matrix('../lab1/matrices/fem_1210_16x16.txt')
matrix_small

array([[0.11111111, 0.05555556, 0.        , 0.        , 0.05555556,
        0.02777778, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.05555556, 0.11111111, 0.        , 0.        , 0.02777778,
        0.05555556, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.11111111, 0.05555556, 0.        ,
        0.        , 0.05555556, 0.02777778, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05555556, 0.11111111, 0.        ,
        0.        , 0.02777778, 0.05555556, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.05555556, 0.02777778, 0.        , 0.        , 0.11111111,
        0.05555556, 0.        , 

## Faktoryzacja Cholesky'ego i dopełnienie Schura 

wersja z wykładu:

In [174]:
def schur_complement(matrix,  m=0):
    A = matrix.copy()
    n = A.shape[0]
    
    for k in range(n-m):
        if abs(A[k, k]) < 1e-8:
            raise ValueError('singular matrix')
            
        vk = A[k+1:n, k]
        dkk = A[k, k]
        A[k+1:n, k] /= dkk
        
        for j in range(k+1, n):
            A[j:n, j] -= A[j:n, k]*vk[n-j-1]
        
    return A

wersja z internetu (działająca):

In [175]:
def schur_complement2(matrix, m=0):
    S = matrix.copy()
    n = S.shape[0]
    L = np.zeros((n, n))
    
    for j in range(n-m):
        L[j, j] = S[j, j]**0.5
        L[j+1:n, j] = S[j+1:n, j]/L[j,j]
        
        for i in range(j+1, n):
            S[i, j+1:n] -= L[i, j]*L[j+1:n, j]
    
    return L

In [176]:
m = np.array([[1, 2, 3, 4], [2, 30, 6, 1], [3, 6, 50, 2], [4, 1, 2, 100]], dtype=float)

In [177]:
print(schur_complement(m, 0))
print(schur_complement2(m, 0))
print(np.linalg.cholesky(m))

[[ 1.          2.          3.          4.        ]
 [ 2.         22.          6.          1.        ]
 [ 3.         -0.27272727 40.81404959  2.        ]
 [ 4.         -0.68181818 -0.25640377 91.7483067 ]]
[[ 1.          0.          0.          0.        ]
 [ 2.          5.09901951  0.          0.        ]
 [ 3.          0.          6.40312424  0.        ]
 [ 4.         -1.37281295 -1.56173762  8.92616156]]
[[ 1.          0.          0.          0.        ]
 [ 2.          5.09901951  0.          0.        ]
 [ 3.          0.          6.40312424  0.        ]
 [ 4.         -1.37281295 -1.56173762  8.92616156]]


W celu weryfikacji poprawności powyższej funkcji, porównujemy wynik z faktoryzacją otrzymaną przy użyciu funkcji biblioteki numpy: 

In [178]:
def cholesky_test(A):
    return np.allclose(schur_complement2(A, 0), np.linalg.cholesky(A))

In [179]:
cholesky_test(matrix_small)

True

### Pomiar czasów

"(...) proszę narysować następujący wykres: <br>
> oś pozioma: rozmiar macierzy n dla liczby przedziałów nxx=2,3,4,… (tak duże macierze ile się uda policzyć na laptopie), <br>
oś pionowa: czas [s] (Octave tic; Schur Complement(...); toc)  

Proszę narysować 

> a) wykres czasu obliczeń dopełnień Schura o rozmiarze n/2
 ** b) wykres czasu obliczeń dopełnień Schura o rozmiarze n/4
 ** c) … takie podziały jakie mają sens do rozmiaru 1"

In [180]:
def compare_times(A):
    pass

* dla macierzy IGA riga=0, pxx=2, rxx=0

* dla macierzy FEM riga=1, pxx=2, rxx=0

## Koszt obliczeniowy i pamięciowy

"3. Jaki jest koszt obliczeniowy i pamięciowy (flopsy i memopsy) zaimplementowanego
algorytmu?"

## Wnioski

* 
* 

M. Hawryluk, N. Wolniak. 2021